In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
data = pd.read_csv('../../data/vehicle_claims_labeled.csv')

In [3]:
data.dtypes

Maker                 object
 Genmodel             object
 Genmodel_ID          object
Adv_year               int64
Adv_month              int64
Color                 object
Reg_year             float64
Bodytype              object
Runned_Miles           int64
Engin_size            object
Gearbox               object
Fuel_type             object
Price                float64
Seat_num             float64
Door_num             float64
issue                 object
issue_id               int64
Adv_day                int64
breakdown_date        object
repair_complexity      int64
repair_cost          float64
repair_hours         float64
Label                  int64
category_anomaly       int64
repair_date           object
dtype: object

In [4]:
numerical_cols = ['Price', 'Seat_num', 'Door_num', 'repair_cost', 'repair_hours']

In [5]:
som_train_data = data[numerical_cols]

In [6]:
y = data['Label']

In [7]:
y = data['Label']
count = 0
for i in y:
    if i == 1:
        count += 1
print(count)

56749


In [8]:
from sklearn.preprocessing import MinMaxScaler
from minisom import MiniSom

In [9]:
sc = MinMaxScaler (feature_range = (0,1))
som_train_data = sc.fit_transform(som_train_data)
pd.DataFrame(som_train_data)

,0,1,2,3,4
0,0.067387,0.210526,0.571429,0.003055,0.000230
1,0.068065,0.210526,0.571429,0.003395,0.000251
2,0.068182,0.210526,0.571429,0.003706,0.000251
3,0.068644,0.210526,0.571429,0.003070,0.000230
4,0.067860,0.210526,0.571429,0.003122,0.000251
...,...,...,...,...,...
268250,0.066196,0.052632,0.000000,0.003823,0.000348
268251,0.066125,0.947368,0.000000,0.003427,0.000334
268252,0.067990,0.052632,0.000000,0.003303,0.000251
268253,0.068644,0.052632,0.000000,0.006949,0.000376


In [10]:
som = MiniSom(x= 10, y= 10, input_len=5, sigma=1, learning_rate=0.05)
som.random_weights_init(som_train_data)

In [11]:
som.train_random(som_train_data, 100000)

In [12]:
outlier_percentage = 56749/268255

In [13]:
quantization_errors = np.linalg.norm(som.quantization(som_train_data) - som_train_data, axis=1)

In [14]:
error_threshold = np.percentile(quantization_errors, 
                               100*(1-outlier_percentage)+5)

In [15]:
is_anomaly = quantization_errors > error_threshold

In [16]:
label = []
for i in range(len(is_anomaly)):
    if is_anomaly[i] == True:
        label.append(1)
    else:
        label.append(0)

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y, label, average='binary')
recall = recall_score(y, label, average='binary')
f1 = f1_score(y, label, average='binary')

In [18]:
precision, recall, f1

(0.512725846274546, 0.391548749757705, 0.4440181443958196)

In [19]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y, label).ravel()
tn, fp, fn, tp

(190389, 21117, 34529, 22220)

In [20]:
categorical_cols = ['Maker', ' Genmodel', 'Color', 'Bodytype', 'Engin_size', 'Gearbox', 'Fuel_type',
                        'Seat_num', 'Door_num', 'issue', 'issue_id', 'repair_complexity']

In [30]:
from utils import *
cols_to_remove = [' Genmodel_ID', 'Adv_year', 'Adv_month', 'Adv_day', 'breakdown_date', 'repair_date', 'category_anomaly']
train_data = remove_cols(data, cols_to_remove)
train_data = label_encoding(train_data, categorical_cols)

In [31]:
sc = MinMaxScaler (feature_range = (0,1))
train_data = sc.fit_transform(train_data)
pd.DataFrame(train_data)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.090090,0.012532,0.067387,0.003055,0.000230,0.0,0.057471,0.170950,0.863636,0.833333,0.932432,0.000000,0.769231,0.210526,0.5,0.357143,0.0,0.666667
1,0.091892,0.010025,0.068065,0.003395,0.000251,0.0,0.057471,0.170950,0.409091,0.833333,0.932432,0.000000,0.769231,0.210526,0.5,0.428571,0.0,0.666667
2,0.091892,0.011749,0.068182,0.003706,0.000251,0.0,0.057471,0.170950,0.090909,0.833333,0.932432,0.000000,0.769231,0.210526,0.5,0.500000,0.0,0.666667
3,0.092793,0.005326,0.068644,0.003070,0.000230,0.0,0.057471,0.170950,0.363636,0.833333,0.932432,0.000000,0.769231,0.210526,0.5,1.000000,0.0,0.666667
4,0.092793,0.012791,0.067860,0.003122,0.000251,0.0,0.057471,0.170950,0.409091,0.833333,0.932432,0.000000,0.769231,0.210526,0.5,0.071429,0.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268250,0.095495,0.003414,0.066196,0.003823,0.000348,1.0,0.988506,0.815642,1.000000,0.222222,0.310811,0.666667,0.769231,0.052632,0.0,0.214286,0.8,0.333333
268251,0.095495,0.003447,0.066125,0.003427,0.000334,1.0,0.988506,0.815642,1.000000,0.222222,1.000000,0.666667,0.692308,0.947368,0.0,0.928571,0.0,0.333333
268252,0.104505,0.003133,0.067990,0.003303,0.000251,1.0,1.000000,0.347486,0.818182,0.222222,0.283784,0.666667,0.769231,0.052632,0.0,0.785714,0.4,0.333333
268253,0.104505,0.003373,0.068644,0.006949,0.000376,1.0,1.000000,0.347486,0.363636,0.222222,0.283784,0.666667,0.769231,0.052632,0.0,0.214286,0.6,0.333333


In [34]:
som = MiniSom(x= 10, y= 10, input_len=18, sigma=1, learning_rate=0.05)
som.random_weights_init(train_data)
som.train_random(train_data, 100000)

In [35]:
quantization_errors = np.linalg.norm(som.quantization(train_data) - train_data, axis=1)
error_threshold = np.percentile(quantization_errors, 
                               100*(1-outlier_percentage)+5)
is_anomaly = quantization_errors > error_threshold

In [36]:
label = []
for i in range(len(is_anomaly)):
    if is_anomaly[i] == True:
        label.append(1)
    else:
        label.append(0)

In [37]:
precision = precision_score(y, label, average='binary')
recall = recall_score(y, label, average='binary')
f1 = f1_score(y, label, average='binary')

In [38]:
precision, recall, f1

(0.478044165493689, 0.36506370156302315, 0.41398397378254703)

In [22]:
outlier_percentage = 0.21
som = MiniSom(x= 10, y= 10, input_len=5, sigma=1, learning_rate=0.05)
som.random_weights_init(som_train_data)
som.train_random(som_train_data, 100000)
quantization_errors = np.linalg.norm(som.quantization(som_train_data) - som_train_data, axis=1)
error_threshold = np.percentile(quantization_errors, 
                               100*(1-outlier_percentage)+5)
is_anomaly = quantization_errors > error_threshold